# 05 — Flujo Agentico Completo

**Objetivo**: Construir un agente completo "Experto en Comics" con 4 herramientas, router, especialistas, y validador. Multi-step reasoning con trazabilidad completa.

## Contenido
1. Herramientas del agente
2. Arquitectura completa del grafo
3. Multi-step reasoning demo
4. Trazabilidad: cada nodo, tool, costo

In [ ]:
import os
import json
import time
from typing import TypedDict, Annotated, Literal
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel, Field
import chromadb

load_dotenv()

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

print("=" * 60)
print("FLUJO AGENTICO COMPLETO — Experto en Comics")
print("=" * 60)

In [ ]:
# ============================================================
# SETUP: Cargar datos en ChromaDB
# ============================================================

def cargar_comics(ruta: str) -> list[dict]:
    with open(ruta) as f:
        return json.load(f)

from langchain_text_splitters import RecursiveCharacterTextSplitter

batman_comics = cargar_comics("../data/batman_comics.json")
spider_comics = cargar_comics("../data/spiderman_comics.json")

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

chroma_client = chromadb.Client()

# Crear colecciones separadas por personaje
for personaje, comics in [("batman", batman_comics), ("spiderman", spider_comics)]:
    chunks, metas = [], []
    for comic in comics:
        for i, chunk in enumerate(splitter.split_text(comic["contenido"])):
            chunks.append(chunk)
            metas.append({"personaje": personaje, "arco": comic["arco"], "tema": comic["tema"], "titulo": comic["titulo"], "doc_id": comic["id"]})
    
    embs = embeddings_model.embed_documents(chunks)
    col = chroma_client.create_collection(name=f"comics_{personaje}", metadata={"hnsw:space": "cosine"})
    col.add(ids=[f"{personaje}_{i}" for i in range(len(chunks))], embeddings=embs, documents=chunks, metadatas=metas)
    print(f"Coleccion comics_{personaje}: {col.count()} chunks")

## 1. Herramientas del Agente

In [ ]:
# ============================================================
# HERRAMIENTAS DEL AGENTE
# ============================================================

@tool
def buscar_batman(query: str) -> str:
    """Busca informacion sobre Batman en la base de datos de comics.
    
    Args:
        query: Pregunta o tema a buscar sobre Batman.
    """
    col = chroma_client.get_collection("comics_batman")
    emb = embeddings_model.embed_query(query)
    results = col.query(query_embeddings=[emb], n_results=3, include=["documents", "metadatas"])
    
    if not results["documents"][0]:
        return "No se encontro informacion sobre Batman para esta consulta."
    
    contexto = []
    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        contexto.append(f"[{meta['titulo']} - {meta['arco']}]: {doc}")
    return "\n\n".join(contexto)


@tool
def buscar_spiderman(query: str) -> str:
    """Busca informacion sobre Spider-Man en la base de datos de comics.
    
    Args:
        query: Pregunta o tema a buscar sobre Spider-Man.
    """
    col = chroma_client.get_collection("comics_spiderman")
    emb = embeddings_model.embed_query(query)
    results = col.query(query_embeddings=[emb], n_results=3, include=["documents", "metadatas"])
    
    if not results["documents"][0]:
        return "No se encontro informacion sobre Spider-Man para esta consulta."
    
    contexto = []
    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        contexto.append(f"[{meta['titulo']} - {meta['arco']}]: {doc}")
    return "\n\n".join(contexto)


@tool
def comparar_heroes(aspecto: str) -> str:
    """Compara Batman y Spider-Man buscando informacion de ambos.
    
    Args:
        aspecto: Aspecto a comparar (fuerza, filosofia, equipo, relaciones, etc).
    """
    batman_col = chroma_client.get_collection("comics_batman")
    spider_col = chroma_client.get_collection("comics_spiderman")
    
    emb = embeddings_model.embed_query(aspecto)
    
    batman_results = batman_col.query(query_embeddings=[emb], n_results=2, include=["documents", "metadatas"])
    spider_results = spider_col.query(query_embeddings=[emb], n_results=2, include=["documents", "metadatas"])
    
    contexto = "BATMAN:\n"
    for doc, meta in zip(batman_results["documents"][0], batman_results["metadatas"][0]):
        contexto += f"[{meta['arco']}]: {doc[:300]}\n"
    
    contexto += "\nSPIDER-MAN:\n"
    for doc, meta in zip(spider_results["documents"][0], spider_results["metadatas"][0]):
        contexto += f"[{meta['arco']}]: {doc[:300]}\n"
    
    return contexto


@tool
def calcular_estadisticas(personaje: str) -> str:
    """Calcula estadisticas y nivel de poder de un personaje.
    
    Args:
        personaje: Nombre del personaje (batman o spiderman).
    """
    stats = {
        "batman": {
            "fuerza": 35, "inteligencia": 95, "tecnologia": 90, "combate": 85,
            "liderazgo": 90, "preparacion": 98, "recursos": 95, "total_promedio": 84
        },
        "spiderman": {
            "fuerza": 70, "inteligencia": 85, "agilidad": 95, "sentido_aracnido": 90,
            "creatividad": 88, "resistencia": 75, "inventiva": 80, "total_promedio": 83
        },
    }
    s = stats.get(personaje.lower())
    if s:
        return json.dumps(s, indent=2, ensure_ascii=False)
    return f"No hay estadisticas para {personaje}"


agent_tools = [buscar_batman, buscar_spiderman, comparar_heroes, calcular_estadisticas]

print("Herramientas del agente:")
for t in agent_tools:
    print(f"  - {t.name}: {t.description.split(chr(10))[0]}")

## 2. Arquitectura del Grafo

```
                    ┌──────────┐
                    │  START    │
                    └────┬─────┘
                         │
                    ┌────▼─────┐
              ┌─────│  Agent   │─────┐
              │     └────┬─────┘     │
              │          │           │
         tool_calls    no tools   tool_calls
              │          │           │
         ┌────▼────┐     │     ┌────▼────┐
         │  Tools  │     │     │  Tools  │
         └────┬────┘     │     └────┬────┘
              │          │          │
              └──────────┼──────────┘
                         │
                    ┌────▼─────┐
                    │ Validar  │
                    └────┬─────┘
                         │
                    ┌────▼─────┐
                    │   END    │
                    └──────────┘
```

In [ ]:
# ============================================================
# GRAFO DEL AGENTE COMPLETO
# ============================================================

class ExpertState(TypedDict):
    messages: Annotated[list, add_messages]
    trace: list[dict]


SYSTEM_PROMPT = """Eres un experto en comics de Batman y Spider-Man. Tu trabajo es responder 
preguntas usando las herramientas disponibles para buscar informacion precisa.

REGLAS:
1. Siempre busca informacion antes de responder
2. Para preguntas de comparacion, busca en AMBAS bases de datos
3. Cita las fuentes (arcos narrativos) en tu respuesta
4. Responde en español
5. Se preciso y evita inventar informacion

HERRAMIENTAS:
- buscar_batman: buscar en la base de datos de Batman
- buscar_spiderman: buscar en la base de datos de Spider-Man
- comparar_heroes: buscar en ambas bases para comparar
- calcular_estadisticas: obtener stats numericas de un personaje
"""

llm_with_tools = llm.bind_tools(agent_tools)


def nodo_agente(state: ExpertState) -> dict:
    """Nodo principal del agente."""
    messages = [SystemMessage(content=SYSTEM_PROMPT)] + state["messages"]
    response = llm_with_tools.invoke(messages)
    
    trace_entry = {
        "nodo": "agent",
        "timestamp": time.time(),
        "tool_calls": len(response.tool_calls) if response.tool_calls else 0,
        "has_content": bool(response.content),
    }
    
    return {
        "messages": [response],
        "trace": state.get("trace", []) + [trace_entry],
    }


def should_continue(state: ExpertState) -> str:
    last = state["messages"][-1]
    if hasattr(last, "tool_calls") and last.tool_calls:
        return "tools"
    return "validar"


def nodo_validar(state: ExpertState) -> dict:
    """Validacion final de la respuesta."""
    last_content = state["messages"][-1].content if state["messages"] else ""
    trace_entry = {
        "nodo": "validar",
        "timestamp": time.time(),
        "respuesta_length": len(last_content) if last_content else 0,
    }
    return {"trace": state.get("trace", []) + [trace_entry]}


def tool_node_with_trace(state: ExpertState) -> dict:
    """ToolNode wrapper que agrega trazabilidad."""
    tool_node = ToolNode(agent_tools)
    result = tool_node.invoke(state)
    
    trace_entry = {
        "nodo": "tools",
        "timestamp": time.time(),
        "tools_executed": len(result.get("messages", [])),
    }
    
    result_trace = state.get("trace", []) + [trace_entry]
    if "trace" not in result:
        result["trace"] = result_trace
    
    return result


# Build graph
graph = StateGraph(ExpertState)
graph.add_node("agent", nodo_agente)
graph.add_node("tools", tool_node_with_trace)
graph.add_node("validar", nodo_validar)

graph.add_edge(START, "agent")
graph.add_conditional_edges("agent", should_continue, {
    "tools": "tools",
    "validar": "validar",
})
graph.add_edge("tools", "agent")
graph.add_edge("validar", END)

expert_app = graph.compile()
print("Agente Experto en Comics compilado")

## 3. Multi-Step Reasoning Demo

In [ ]:
# ============================================================
# DEMO: Multi-step reasoning
# ============================================================

queries = [
    "Quien seria mejor lider en una crisis: Batman o Spider-Man? Justifica con evidencia de los comics.",
    "Compara la filosofia de Batman con la de Spider-Man. Que los diferencia fundamentalmente?",
    "Que rol juega la tecnologia en Batman vs los poderes innatos de Spider-Man?",
]

print("=" * 60)
print("MULTI-STEP REASONING")
print("=" * 60)

for query in queries:
    print(f"\n{'='*60}")
    print(f"QUERY: {query}")
    print("=" * 60)
    
    t0 = time.time()
    result = expert_app.invoke({
        "messages": [HumanMessage(content=query)],
        "trace": [],
    })
    total_time = (time.time() - t0) * 1000
    
    # Trace
    print(f"\nTRAZABILIDAD:")
    for entry in result.get("trace", []):
        print(f"  [{entry['nodo']:10s}] {json.dumps({k: v for k, v in entry.items() if k != 'nodo'})}")
    
    # Count messages by type
    msg_types = {}
    for msg in result["messages"]:
        t = msg.__class__.__name__
        msg_types[t] = msg_types.get(t, 0) + 1
    print(f"\nMensajes: {msg_types}")
    print(f"Latencia total: {total_time:.0f}ms")
    
    # Final response
    respuesta = result["messages"][-1].content if result["messages"][-1].content else "(sin contenido)"
    print(f"\nRESPUESTA:\n{respuesta[:500]}...")

## 4. Analisis de Trazabilidad

In [ ]:
# ============================================================
# ANALISIS DE COSTOS POR QUERY
# ============================================================

print("=" * 60)
print("RESUMEN DE TRAZABILIDAD")
print("=" * 60)

# Contar operaciones por query
for i, query in enumerate(queries):
    result = expert_app.invoke({
        "messages": [HumanMessage(content=query)],
        "trace": [],
    })
    
    trace = result.get("trace", [])
    agent_calls = sum(1 for t in trace if t["nodo"] == "agent")
    tool_calls = sum(1 for t in trace if t["nodo"] == "tools")
    
    print(f"\nQuery {i+1}: {query[:50]}...")
    print(f"  Iteraciones del agente: {agent_calls}")
    print(f"  Llamadas a herramientas: {tool_calls}")
    print(f"  Total nodos ejecutados: {len(trace)}")

## Takeaways

1. Un agente completo combina **routing, herramientas, validacion y trazabilidad**
2. Para preguntas de comparacion, el agente automaticamente busca en **ambas bases de datos**
3. La trazabilidad permite entender **exactamente que hizo el agente** en cada paso
4. Multi-step reasoning requiere multiples tool calls, lo que incrementa costo y latencia
5. El pattern `agent → tools → agent → ... → validar` es el standard de LangGraph
6. En produccion, la trazabilidad es esencial para debugging y auditoria